In [1]:
!pip install -q transformers torchvision
print("1")
!pip install -q git+https://github.com/huggingface/peft.git
print("2")
!pip install -q transformers accelerate bitsandbytes torch torchvision
print("3")
!pip install -q git+https://github.com/huggingface/transformers.git
print("4")

1
2
3
4


In [2]:
!pip install diffusers
!pip install opencv-python

# Imports

In [2]:
import os
import cv2
import numpy as np
from typing import List, Union
import cv2
import numpy as np
from typing import Optional, Tuple, List, Union
import os
import torch
import numpy as np
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, AutoencoderKL
from diffusers.utils import load_image
import cv2
from typing import List, Union, Optional, Dict, Any
import matplotlib.pyplot as plt
import torch
from torchvision import models
import torch.nn as nn
from typing import List, Tuple, Iterator
from pathlib import Path
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [3]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
captionmodel = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda" if torch.cuda.is_available() else "cpu")

# Image container

In [4]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
captionmodel = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class ImageContainer:
    def __init__(self, image_path: str, label: str):
        self.image_path = image_path
        self.label = label
        self.original_image = self._read_image(image_path)
        self.variants = []
        self.color_maps = []
        self.caption = self.caption_image(image_path)
        self.predicted_variants_labels = []

    def _read_image(self, path: str) -> np.ndarray:
        """Reads and validates the image from the given path."""
        supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
        # Validate path
        if not os.path.isfile(path):
            raise FileNotFoundError(f"The file '{path}' does not exist.")
        if not path.lower().endswith(supported_extensions):
            raise ValueError(f"Unsupported file type. Supported types are: {supported_extensions}")

        # Read image
        image = cv2.imread(path)
        if image is None:
            raise ValueError(f"Failed to read the image from path: '{path}'.")

        # Convert BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Optional: Validate image dtype and shape
        if image.dtype != np.uint8:
            raise TypeError(f"Unsupported image dtype '{image.dtype}'. Expected 'np.uint8'.")
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError(f"Expected a 3-channel RGB image, but got shape {image.shape}.")

        return image

    def caption_image(self, path: str) -> str:
        """
        Generate a caption for the image using the BLIP model.

        Returns:
            str: The generated caption.
        """
        image = Image.open(path).convert('RGB')
        #display(image)
        inputs = processor(images=image, return_tensors="pt").to(captionmodel.device)
        out = captionmodel.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        return caption

    def add_color_condition(self, canny_img_condition: np.ndarray = None, num_blocks: int = 4):
        """
        Add a color conditioning image to the variants list.
        If `canny_img_condition` is None, generate a new one with `num_blocks` color blocks.

        Parameters:
            canny_img_condition (np.ndarray): Optional RGB conditioning image.
            num_blocks (int): Number of blocks to generate if image is None.
        """
        if self.original_image.shape[:2] != canny_img_condition.shape[:2]:
            raise ValueError("The color-conditioned image must be the same size as the original image.")

        self.variants.append(canny_img_condition)

    def add_variants(self, images: Union[np.ndarray, List[np.ndarray]], validate: bool = True) -> int:
        """
        Add one or more variant images to the variants list.
        Validates that each image is RGB and has the same dimensions as the original image.

        Parameters:
            images (Union[np.ndarray, List[np.ndarray]]): Single image or list of images to add
            validate (bool): Whether to validate image format and dimensions

        Returns:
            int: Number of images successfully added

        Raises:
            ValueError: If any image fails the validation checks
        """
        # Convert single image to list for uniform processing
        if not isinstance(images, list):
            images = [images]

        added_count = 0

        for idx, img in enumerate(images):
            if validate:
                # Check if image is None
                if img is None:
                    raise ValueError(f"Image at index {idx} is None")

                # Check if image is numpy array
                if not isinstance(img, np.ndarray):
                    raise TypeError(f"Image at index {idx} is not a numpy array")

                # Check if image has the same shape as original
                # if img.shape[:2] != self.original_image.shape[:2]:
                #     raise ValueError(f"Image at index {idx} has dimensions {img.shape[:2]}, "
                #                      f"expected {self.original_image.shape[:2]}")

                # Check if image is RGB (3 channels)
                if len(img.shape) != 3 or img.shape[2] != 3:
                    raise ValueError(f"Image at index {idx} has {img.shape[2] if len(img.shape) == 3 else 0} channels, "
                                     f"expected 3 (RGB)")

                # Check dtype
                if img.dtype != np.uint8:
                    raise TypeError(f"Image at index {idx} has dtype {img.dtype}, expected np.uint8")

            # Add image to variants
            self.variants.append(img)
            added_count += 1

        return added_count

    def extend_variants(self, images: List[np.ndarray], validate: bool = True) -> int:
        """
        Wrapper around add_variants for better semantic clarity when extending with multiple images.

        Parameters:
            images (List[np.ndarray]): List of images to add
            validate (bool): Whether to validate image format and dimensions

        Returns:
            int: Number of images successfully added
        """
        return self.add_variants(images, validate)

    def add_color_maps(self, color_maps: Union[np.ndarray, List[np.ndarray]], validate: bool = True) -> int:
        """
        Add one or more color maps to the color_maps list.
        Validates that each color map is compatible with the original image.

        Parameters:
            color_maps (Union[np.ndarray, List[np.ndarray]]): Single color map or list of color maps to add
            validate (bool): Whether to validate color map format and dimensions

        Returns:
            int: Number of color maps successfully added

        Raises:
            ValueError: If any color map fails the validation checks
        """
        # Convert single color map to list for uniform processing
        if not isinstance(color_maps, list):
            color_maps = [color_maps]

        added_count = 0

        for idx, cmap in enumerate(color_maps):
            if validate:
                # Check if color map is None
                if cmap is None:
                    raise ValueError(f"Color map at index {idx} is None")

                # Check if color map is numpy array
                if not isinstance(cmap, np.ndarray):
                    raise TypeError(f"Color map at index {idx} is not a numpy array")

                # Check if color map has the same height and width as original image
                if cmap.shape[:2] != self.original_image.shape[:2]:
                    raise ValueError(f"Color map at index {idx} has dimensions {cmap.shape[:2]}, "
                                     f"expected {self.original_image.shape[:2]}")

                # Check dtype
                if cmap.dtype != np.uint8:
                    raise TypeError(f"Color map at index {idx} has dtype {cmap.dtype}, expected np.uint8")

            # Add color map to color_maps list
            self.color_maps.append(cmap)
            added_count += 1

        return added_count

    def extend_color_maps(self, color_maps: List[np.ndarray], validate: bool = True) -> int:
        """
        Wrapper around add_color_maps for better semantic clarity when extending with multiple color maps.

        Parameters:
            color_maps (List[np.ndarray]): List of color maps to add
            validate (bool): Whether to validate color map format and dimensions

        Returns:
            int: Number of color maps successfully added
        """
        return self.add_color_maps(color_maps, validate)

# ColorCannyConditionCreator

In [6]:
import cv2
import numpy as np
from typing import Tuple, List
from PIL import Image

class ColorCannyConditionCreator:
    def __init__(self, image: Image):

        self.original_image = image.original_image

    def load_image(self, image_path: str) -> None:
        self.original_image = cv2.imread(image_path)
        if self.original_image is None:
            raise ValueError(f"Could not load image from {image_path}")
        self.original_image = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB)

    def generate_canny_edges(self, low_threshold: int = 100, high_threshold: int = 200) -> np.ndarray:

        if self.original_image is None:
            raise ValueError("No image loaded. Call load_image() first.")

        gray = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(gray, low_threshold, high_threshold)
        # Convert back to RGB for compatibility with the fusion process
        edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        return edges_rgb

    def generate_color_condition_image(self, color: Tuple[int, int, int] = (0, 0, 255)) -> np.ndarray:

        if self.original_image is None:
            raise ValueError("No image loaded. Call load_image() first.")

        height, width = self.original_image.shape[:2]
        image = np.zeros((height, width, 3), dtype=np.uint8)
        image[:] = color
        return image

    def generate_color_condition_image_random(self, num_blocks: int = 4) -> np.ndarray:

        if self.original_image is None:
            raise ValueError("No image loaded. Call load_image() first.")

        height, width = self.original_image.shape[:2]
        block_configs = {2: (1, 2), 4: (2, 2), 6: (2, 3)}

        if num_blocks not in block_configs:
            raise ValueError("Supported num_blocks values are 2, 4, or 6.")

        rows, cols = block_configs[num_blocks]
        block_h, block_w = height // rows, width // cols
        image = np.zeros((height, width, 3), dtype=np.uint8)

        for i in range(rows):
            for j in range(cols):
                y_start, y_end = i * block_h, (i + 1) * block_h
                x_start, x_end = j * block_w, (j + 1) * block_w
                # Generate random RGB color
                color = tuple(np.random.randint(0, 256, size=3))
                image[y_start:y_end, x_start:x_end] = color

        return image

    def generate_brightness_condition(self, brightness_level: float = 0.5) -> np.ndarray:

        if self.original_image is None:
            raise ValueError("No image loaded. Call load_image() first.")

        brightness_level = max(0.0, min(1.0, brightness_level))  # Clamp between 0 and 1
        height, width = self.original_image.shape[:2]

        # Create a uniform brightness image
        brightness_value = int(brightness_level * 255)
        image = np.ones((height, width, 3), dtype=np.uint8) * brightness_value

        return image

    def compress_image_to_color_map(self, target_rows: int = 2, target_cols: int = 3) -> np.ndarray:
        """
        Compress the original image to a smaller grid and extract dominant colors.
        Returns a 2D array of RGB colors for each block.
        """
        if self.original_image is None:
            raise ValueError("No image loaded. Call load_image() first.")

        height, width = self.original_image.shape[:2]
        block_h, block_w = height // target_rows, width // target_cols

        color_map = np.zeros((target_rows, target_cols, 3), dtype=np.uint8)

        for i in range(target_rows):
            for j in range(target_cols):
                y_start = i * block_h
                y_end = min((i + 1) * block_h, height)
                x_start = j * block_w
                x_end = min((j + 1) * block_w, width)

                # Extract the block
                block = self.original_image[y_start:y_end, x_start:x_end]

                # Calculate the average color of the block
                avg_color = np.mean(block.reshape(-1, 3), axis=0).astype(np.uint8)
                color_map[i, j] = avg_color

        return color_map

    def create_color_map_image(self, color_map: np.ndarray) -> np.ndarray:
        """
        Create a full-size image using the compressed color map.
        """
        if self.original_image is None:
            raise ValueError("No image loaded. Call load_image() first.")

        height, width = self.original_image.shape[:2]
        target_rows, target_cols = color_map.shape[:2]

        block_h, block_w = height // target_rows, width // target_cols
        image = np.zeros((height, width, 3), dtype=np.uint8)

        for i in range(target_rows):
            for j in range(target_cols):
                y_start = i * block_h
                y_end = min((i + 1) * block_h, height)
                x_start = j * block_w
                x_end = min((j + 1) * block_w, width)

                image[y_start:y_end, x_start:x_end] = color_map[i, j]

        return image

    def apply_color_variations(self, color_map: np.ndarray, delta: int = 20) -> List[np.ndarray]:
        """
        Create three variations of the color map: negative delta, original, positive delta.
        """
        variations = []

        # Negative delta
        negative_map = np.clip(color_map.astype(np.int16) - delta, 0, 255).astype(np.uint8)
        variations.append(negative_map)

        # Original (no change)
        variations.append(color_map.copy())

        # Positive delta
        positive_map = np.clip(color_map.astype(np.int16) + delta, 0, 255).astype(np.uint8)
        variations.append(positive_map)

        return variations

    def fuse_color_and_canny(
        self,
        color_image: np.ndarray,
        canny_image: np.ndarray,
        alpha: float = 0.7
    ) -> np.ndarray:

        if color_image.shape != canny_image.shape:
            raise ValueError("Color image and Canny image must have the same dimensions.")

        # Convert to float for blending
        color_float = color_image.astype(float)
        canny_float = canny_image.astype(float)

        # Blend images
        fused = cv2.addWeighted(color_float, alpha, canny_float, 1.0 - alpha, 0)

        # Convert back to uint8
        fused_uint8 = np.clip(fused, 0, 255).astype(np.uint8)

        return fused_uint8

    def create_full_conditioning(
        self,
        condition_type: str = "solid_color",
        color: Tuple[int, int, int] = (0, 0, 255),
        num_blocks: int = 4,
        brightness_level: float = 0.5,
        canny_low: int = 100,
        canny_high: int = 200,
        blend_alpha: float = 0.7
    ) -> np.ndarray:

        # Generate Canny edge detection
        canny = self.generate_canny_edges(canny_low, canny_high)

        # Generate color condition based on type
        if condition_type == "solid_color":
            color_cond = self.generate_color_condition_image(color)
        elif condition_type == "random_blocks":
            color_cond = self.generate_color_condition_image_random(num_blocks)
        elif condition_type == "brightness":
            color_cond = self.generate_brightness_condition(brightness_level)
        else:
            raise ValueError("Unsupported condition type. Use 'solid_color', 'random_blocks', or 'brightness'")

        # Fuse color and Canny
        fused = self.fuse_color_and_canny(color_cond, canny, blend_alpha)

        return fused

    def generate_multiple_conditions(
        self,
        canny_low: int = 100,
        canny_high: int = 200,
        blend_alpha: float = 0.7,
        color_delta: int = 20,
        target_rows: int = 2,
        target_cols: int = 3
    ) -> List[np.ndarray]:
        """
        Generate multiple conditioning images using compressed color map approach.

        Steps:
        1. Generate Canny edges from the original image
        2. Compress the image to a 2x3 (or custom) grid and extract colors
        3. Create 3 color variations (negative delta, original, positive delta)
        4. Create full-size color maps and fuse with Canny edges

        Returns:
            List of 3 fused images with color variations
        """
        if self.original_image is None:
            raise ValueError("No image loaded. Call load_image() first.")

        # Step 1: Generate Canny edges
        canny_edges = self.generate_canny_edges(canny_low, canny_high)

        # Step 2: Compress image to color map
        # Result: color_map is a 2×3×3 array containing 6 RGB colors
        compressed_color_map = self.compress_image_to_color_map(target_rows, target_cols)

        # Step 3: Create color variations
        color_variations = self.apply_color_variations(compressed_color_map, color_delta)

        # Step 4: Create full-size images and fuse with Canny
        result_images = []
        for color_map in color_variations:
            # Create full-size color map image
            color_image = self.create_color_map_image(color_map)

            # Fuse with Canny edges
            fused_image = self.fuse_color_and_canny(color_image, canny_edges, blend_alpha)
            result_images.append(fused_image)

        return result_images

# ColorCannyDiffuser

In [7]:
from PIL import Image as PILImage


class ColorCannyDiffuser:
    """
    A class to handle diffusion with Color-Canny-Controlnet model
    """

    def __init__(self, model_id="ghoskno/Color-Canny-Controlnet-model", device="cuda", local_model_path=None):
        """
        Initialize the Color-Canny-Controlnet diffusion pipeline.

        Args:
            model_id (str): HuggingFace model ID or local path
            device (str): Device to use for inference ("cuda" or "cpu")
            local_model_path (str): Path to local model file (diffusion_pytorch_model.bin)
        """
        self.model_id = model_id
        self.local_model_path = local_model_path
        self.device = device if torch.cuda.is_available() and device == "cuda" else "cpu"
        self.pipe = self._load_pipeline()

    def _load_pipeline(self):
        """Load the diffusion pipeline with the ControlNet model"""
        if self.local_model_path and os.path.exists(self.local_model_path):

            # Load the ControlNet model first
            model_dir = os.path.dirname(self.local_model_path)
            controlnet = ControlNetModel.from_pretrained(
                model_dir,
                local_files_only=True,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
            )

            # Then load a base Stable Diffusion model and create a ControlNet pipeline
            pipe = StableDiffusionControlNetPipeline.from_pretrained(
                "runwayml/stable-diffusion-v1-5",
                controlnet=controlnet,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                safety_checker=None  # 🔧 Disable it right at loading
            )


            # Disable the NSFW safety checker
            pipe = pipe.to(self.device)


            # print("u3", pipe)
            pipe = pipe.to(self.device)
            # print("u4", pipe)
            return pipe


    def generate(self,
                prompt: str,
                input_image=None,
                num_inference_steps: int = 50,
                guidance_scale: float = 7.5,
                seed: Optional[int] = None,
                output_path: Optional[str] = None,
                width: int = 512,
                height: int = 512,
                canny_low_threshold: int = 100,
                canny_high_threshold: int = 200):
        """
        Generate an image using the Color-Canny-Controlnet model

        Args:
            prompt (str): The text prompt to guide generation
            input_image: Path to input image or PIL.Image for conditioning
            num_inference_steps (int): Number of denoising steps
            guidance_scale (float): Guidance scale for classifier-free guidance
            seed (int, optional): Random seed for reproducibility
            output_path (str, optional): Path to save the output image
            width (int): Width of the generated image
            height (int): Height of the generated image
            canny_low_threshold (int): Lower threshold for Canny edge detection
            canny_high_threshold (int): Upper threshold for Canny edge detection

        Returns:
            PIL.Image: Generated image
        """
        # Set the seed if provided
        if seed is not None:
            torch.manual_seed(seed)
            np.random.seed(seed)
            generator = torch.Generator(device=self.device).manual_seed(seed)
        else:
            generator = None

        # Process the input image if provided
        if input_image is not None:
            processed_image = input_image  # Assuming it's already processed
            #processed_image = Image.open(processed_image).convert('RGB')
            # Generate with control image
            if input_image is not None:
                if isinstance(input_image, np.ndarray):
                    processed_image = PILImage.fromarray(input_image.astype('uint8')).convert("RGB")
                elif isinstance(input_image, Image.Image):
                    processed_image = input_image
                else:
                    processed_image = Image.open(input_image).convert("RGB")
            print('Generating from control image')
            generator_params = {
                "prompt": prompt,
                "image": processed_image,
                "num_inference_steps": num_inference_steps,
                "guidance_scale": guidance_scale,
                "generator": generator,
            }
        else:
            print('Generate without control image')
            # Generate without control image
            generator_params = {
                "prompt": prompt,
                "num_inference_steps": num_inference_steps,
                "guidance_scale": guidance_scale,
                "height": height,
                "width": width,
                "generator": generator,
            }

        # Generate the image
        # print(self.pipe)
        output = self.pipe(**generator_params)
        generated_image = output.images[0]

        # Save the output if requested
        if output_path:
            generated_image.save(output_path)
            print(f"Image saved to {output_path}")

        return generated_image

    def visualize_process(self,
                        input_image,
                        prompt: str,
                        num_inference_steps: int = 50,
                        guidance_scale: float = 7.5,
                        seed: Optional[int] = None,
                        canny_low_threshold: int = 100,
                        canny_high_threshold: int = 200):
        """
        Visualize the input, edge detection, and generated image

        Args:
            input_image: Path to input image or PIL.Image
            prompt (str): The text prompt to guide generation
            num_inference_steps (int): Number of denoising steps
            guidance_scale (float): Guidance scale for classifier-free guidance
            seed (int, optional): Random seed for reproducibility
            canny_low_threshold (int): Lower threshold for Canny edge detection
            canny_high_threshold (int): Upper threshold for Canny edge detection
        """
        # Process input image
        original_image = input_image
        #original_image = Image.open(original_image).convert('RGB')

        # Generate the output image
        generated_image = self.generate(
            prompt=prompt,
            input_image=input_image,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            seed=seed,
            canny_low_threshold=canny_low_threshold,
            canny_high_threshold=canny_high_threshold
        )

        # Display the images
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 2, 1)
        plt.title("Original Image")
        plt.imshow(original_image)
        plt.axis('off')



        plt.subplot(1, 2, 2)
        plt.title("Generated Image")
        plt.imshow(generated_image)
        plt.axis('off')

        plt.tight_layout()
        plt.show()

        return original_image, generated_image


    def generate_fused_image(self,
                        input_image,
                        prompt: str,
                        num_inference_steps: int = 50,
                        guidance_scale: float = 7.5,
                        seed: Optional[int] = None,
                        canny_low_threshold: int = 100,
                        canny_high_threshold: int = 200):
        """
        Visualize the input, edge detection, and generated image

        Args:
            input_image: Path to input image or PIL.Image
            prompt (str): The text prompt to guide generation
            num_inference_steps (int): Number of denoising steps
            guidance_scale (float): Guidance scale for classifier-free guidance
            seed (int, optional): Random seed for reproducibility
            canny_low_threshold (int): Lower threshold for Canny edge detection
            canny_high_threshold (int): Upper threshold for Canny edge detection
        """
        # Process input image
        original_image = input_image
        #original_image = Image.open(original_image).convert('RGB')

        # Generate the output image
        generated_image = self.generate(
            prompt=prompt,
            input_image=input_image,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            seed=seed,
            canny_low_threshold=canny_low_threshold,
            canny_high_threshold=canny_high_threshold
        )
        return generated_image




# ImageDataset

In [8]:
class ImageDataset(Dataset):
    """
    PyTorch Dataset for loading images from a directory structure.

    Expected directory structure:
    dataset_path/
    ├── label1/
    │   ├── image1.jpg
    │   ├── image2.png
    │   └── ...
    ├── label2/
    │   ├── image3.jpg
    │   └── ...
    └── ...
    """

    def __init__(self, dataset_path: str, transform=None, return_image_object: bool = False):
        """
        Initialize the ImageDataset.

        Parameters:
            dataset_path (str): Path to the dataset directory
            transform: Optional transform to be applied on a sample
            return_image_object (bool): If True, return Image objects; if False, return (image_array, label) tuples
        """
        self.dataset_path = Path(dataset_path)
        self.valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
        self.image_paths: List[Tuple[str, str]] = []  # (path, label)
        self.transform = transform
        self.return_image_object = return_image_object

        if not self.dataset_path.exists():
            raise FileNotFoundError(f"Dataset path does not exist: {self.dataset_path}")

        self._index_dataset()

    def _index_dataset(self):
        """Index all images in the dataset directory."""
        subdirs = [d for d in self.dataset_path.iterdir() if d.is_dir()]
        if not subdirs:
            raise ValueError(f"No subdirectories (labels) found in {self.dataset_path}")

        subdirs.sort()
        for subdir in subdirs:
            label = subdir.name
            image_files = [
                f for f in subdir.iterdir()
                if f.is_file() and f.suffix.lower() in self.valid_extensions
            ]
            for img_path in image_files:
                self.image_paths.append((str(img_path), label))

        print(f"Indexed dataset with {len(subdirs)} labels and {len(self.image_paths)} total images")

    def __len__(self) -> int:
        """Return the total number of images in the dataset."""
        return len(self.image_paths)

    def __getitem__(self, idx: int):
        """
        Get an item from the dataset.

        Parameters:
            idx (int): Index of the item to retrieve

        Returns:
            If return_image_object is True: ImageContainer object
            If return_image_object is False: tuple of (image_array, label)
        """
        if idx >= len(self.image_paths):
            raise IndexError(f"Index {idx} out of range for dataset of size {len(self.image_paths)}")

        img_path, label = self.image_paths[idx]

        if self.return_image_object:
            # Return ImageContainer object
            image_obj = ImageContainer(image_path=img_path, label=label)
            return image_obj
        else:
            # Return (image_array, label) tuple for standard PyTorch usage
            image = cv2.imread(img_path)
            if image is None:
                raise ValueError(f"Failed to read image: {img_path}")

            # Convert BGR to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Apply transform if provided
            if self.transform:
                image = self.transform(image)

            return image, label

    def get_image_generator(self) -> Iterator[ImageContainer]:
        """
        Generator that yields ImageContainer objects for all images in the dataset.

        Yields:
            ImageContainer: ImageContainer object for each image in the dataset
        """
        for img_path, label in self.image_paths:
            yield ImageContainer(image_path=img_path, label=label)

    def get_labels(self) -> List[str]:
        """
        Get all unique labels in the dataset.

        Returns:
            List[str]: Sorted list of unique labels
        """
        labels = set(label for _, label in self.image_paths)
        return sorted(list(labels))

    def get_label_counts(self) -> dict:
        """
        Get count of images per label.

        Returns:
            dict: Dictionary mapping label to count
        """
        label_counts = {}
        for _, label in self.image_paths:
            label_counts[label] = label_counts.get(label, 0) + 1
        return label_counts

# ImageDisplay

In [9]:

def display_3_images(original_image, canny_image, fused_image):
    """
    Displays the original image, Canny edge image, and fused image side by side.

    Parameters:
        original_image (np.ndarray): The original image.
        canny_image (np.ndarray): The Canny edge image.
        fused_image (np.ndarray): The fused image combining Canny and color map.
    """
    # Create a figure with three subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Display Original Image
    axes[0].imshow(original_image)
    axes[0].set_title("Original Image")
    axes[0].axis("off")  # Hide axes

    # Display Canny Edge Image
    axes[1].imshow(canny_image, cmap='gray')
    axes[1].set_title("Canny Edge Image")
    axes[1].axis("off")  # Hide axes

    # Display Fused Image
    axes[2].imshow(fused_image)
    axes[2].set_title("Fused Image")
    axes[2].axis("off")  # Hide axes

    # Show the plot
    plt.tight_layout()
    plt.show()

def display_2_images(original_image, fused_image):
    """
    Displays the original image, Canny edge image, and fused image side by side.

    Parameters:
        original_image (np.ndarray): The original image.
        fused_image (np.ndarray): The fused image combining Canny and color map.
    """
    # Create a figure with three subplots
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Display Original Image
    axes[0].imshow(original_image)
    axes[0].set_title("Original Image")
    axes[0].axis("off")  # Hide axes

    # Display Canny Edge Image
    axes[1].imshow(fused_image, cmap='gray')
    axes[1].set_title("Fused Image")
    axes[1].axis("off")  # Hide axes

    # Show the plot
    plt.tight_layout()
    plt.show()


# ModelHolder

In [10]:
from typing import Any

class ModelHolder:
    def __init__(self, model: Any, model_name: str = "UnnamedModel"):
        """
        Holds a pre-loaded model.

        Parameters:
            model (Any): A pre-loaded model instance.
            model_name (str): Optional name for identifying the model.
        """
        self.model = model
        self.model_name = model_name
        self.is_trained = hasattr(model, "predict") or hasattr(model, "__call__")

    def predict(self, input_data: Any) -> Any:
        """
        Uses the held model to make a prediction on the given input.

        Parameters:
            input_data (Any): Input data compatible with the model.

        Returns:
            Any: Prediction result.

        Raises:
            ValueError: If the model is not callable or lacks a prediction interface.
        """
        if hasattr(self.model, "predict"):
            return self.model.predict(input_data)
        elif callable(self.model):
            return self.model(input_data)
        else:
            raise ValueError("The held model does not support prediction.")

    def __str__(self):
        return f"ModelHolder(model_name={self.model_name}, model_type={type(self.model).__name__})"


In [11]:
model = models.vgg16(pretrained=True)
model.eval()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## extract_class_names_from_dataset

In [12]:
def extract_class_names_from_dataset(dataset_path: str) -> List[str]:
    """
    Extract class names from dataset directory structure.

    Parameters:
        dataset_path (str): Path to the dataset directory

    Returns:
        List[str]: List of class names found in the dataset directory

    Raises:
        ValueError: If dataset path doesn't exist or no class directories found
    """
    if not os.path.exists(dataset_path):
        raise ValueError(f"Dataset path '{dataset_path}' does not exist")

    # Get all subdirectories (assuming each subdirectory is a class)
    class_names = []
    for item in os.listdir(dataset_path):
        item_path = os.path.join(dataset_path, item)
        if os.path.isdir(item_path):
            class_names.append(item)

    if not class_names:
        raise ValueError(f"No class directories found in '{dataset_path}'")

    # Sort class names for consistency
    class_names.sort()
    return class_names


In [13]:
def extract_class_names_from_dataset_object(dataset_holder) -> List[str]:
    """
    Extract class names from ImageDataset object if it has methods to provide this information.

    Parameters:
        dataset_holder: ImageDataset object

    Returns:
        List[str]: List of class names extracted from the dataset object
    """
    # Check if dataset_holder has a method to get class names
    if hasattr(dataset_holder, 'get_class_names'):
        return dataset_holder.get_class_names()
    elif hasattr(dataset_holder, 'class_names'):
        return dataset_holder.class_names
    elif hasattr(dataset_holder, 'classes'):
        return dataset_holder.classes

    # If no direct method, try to extract from dataset path
    if hasattr(dataset_holder, 'dataset_path'):
        return extract_class_names_from_dataset(dataset_holder.dataset_path)
    elif hasattr(dataset_holder, 'path'):
        return extract_class_names_from_dataset(dataset_holder.path)
    elif hasattr(dataset_holder, 'root'):
        return extract_class_names_from_dataset(dataset_holder.root)

    # Last resort: collect unique labels from all images
    unique_labels = set()
    for image in dataset_holder.get_image_generator():
        if hasattr(image, 'label'):
            label = image.label
            if isinstance(label, (int, np.integer)):
                unique_labels.add(f"class_{label}")
            else:
                unique_labels.add(str(label))

    return sorted(list(unique_labels))

# evaluate_model_robustness

In [14]:
def create_save_directory(class_name: str, base_path: str = "./variants") -> str:
        """Create directory structure for saving variants"""
        save_dir = os.path.join(base_path, f"{class_name}_variants")
        os.makedirs(save_dir, exist_ok=True)
        return save_dir

In [15]:
def save_color_map(color_map_array: np.ndarray, save_path: str, image_idx: int, variant_idx: int) -> str:
        """Save color map to specified path"""
        # Ensure color map is in correct format
        if color_map_array.dtype != np.uint8:
            # Normalize to 0-255 range if needed
            if color_map_array.max() <= 1.0:
                color_map_array = (color_map_array * 255).astype(np.uint8)
            else:
                color_map_array = np.clip(color_map_array, 0, 255).astype(np.uint8)

        # Convert to PIL Image
        if len(color_map_array.shape) == 3:
            if color_map_array.shape[0] in [1, 3, 4]:  # Channel first
                color_map_array = np.transpose(color_map_array, (1, 2, 0))
            pil_image = Image.fromarray(color_map_array)
        else:
            pil_image = Image.fromarray(color_map_array)

        # Create filename
        filename = f"colormap_{image_idx:04d}_variant_{variant_idx:02d}.png"
        full_path = os.path.join(save_path, filename)

        # Save color map
        pil_image.save(full_path)
        return full_path
        """Save generated image to specified path"""
        # Ensure image is in correct format
        if image_array.dtype != np.uint8:
            # Normalize to 0-255 range if needed
            if image_array.max() <= 1.0:
                image_array = (image_array * 255).astype(np.uint8)
            else:
                image_array = np.clip(image_array, 0, 255).astype(np.uint8)

        # Convert to PIL Image
        if len(image_array.shape) == 3:
            if image_array.shape[0] in [1, 3, 4]:  # Channel first
                image_array = np.transpose(image_array, (1, 2, 0))
            pil_image = Image.fromarray(image_array)
        else:
            pil_image = Image.fromarray(image_array)

        # Create filename
        filename = f"image_{image_idx:04d}_variant_{variant_idx:02d}.png"
        full_path = os.path.join(save_path, filename)

        # Save image
        pil_image.save(full_path)
        return full_path

In [16]:
diffuser_model_id: str = "ghoskno/Color-Canny-Controlnet-model"
diffuser_local_path: str = "./diffusion_pytorch_model.bin"
    
print("Initializing diffusion and model components...")

diffuser = ColorCannyDiffuser(
    model_id=diffuser_model_id,
    local_model_path=diffuser_local_path
)
model_holder = ModelHolder(model=model)

The config attributes {'dropout': 0.0, 'sample_size': 32} were passed to ControlNetModel, but are not expected and will be ignored. Please verify your config.json configuration file.


Initializing diffusion and model components...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [17]:
def process_label(label):
        """Convert various label formats to string"""
        return str(label)


In [18]:
def save_generated_image(image_array: np.ndarray, save_path: str, image_idx: int, variant_idx: int) -> str:
        """Save generated image to specified path"""
        # Ensure image is in correct format
        if image_array.dtype != np.uint8:
            # Normalize to 0-255 range if needed
            if image_array.max() <= 1.0:
                image_array = (image_array * 255).astype(np.uint8)
            else:
                image_array = np.clip(image_array, 0, 255).astype(np.uint8)

        # Convert to PIL Image
        if len(image_array.shape) == 3:
            if image_array.shape[0] in [1, 3, 4]:  # Channel first
                image_array = np.transpose(image_array, (1, 2, 0))
            pil_image = Image.fromarray(image_array)
        else:
            pil_image = Image.fromarray(image_array)

        # Create filename
        filename = f"image_{image_idx:04d}_variant_{variant_idx:02d}.png"
        full_path = os.path.join(save_path, filename)

        # Save image
        pil_image.save(full_path)
        return full_path

In [19]:
def save_color_map_np(color_map, colormap_directories, original_label, image_idx, variant_idx, color_maps_base_path):
    try:
        color_map_np = color_map
        if original_label in colormap_directories:
            colormap_path = save_color_map(
                color_map_np,
                colormap_directories[original_label],
                image_idx,
                variant_idx
            )
        else:
            unknown_colormap_dir = create_save_directory(original_label, color_maps_base_path)
            colormap_directories[original_label] = unknown_colormap_dir
            colormap_path = save_color_map(
                color_map_np,
                unknown_colormap_dir,
                image_idx,
                variant_idx
            )
    except Exception as e:
        print(f"Failed to save color map {variant_idx} for image {image_idx}: {e}")


In [20]:
def save_img_variant(original_label, save_directories, image_idx, variant_idx, np_image):
    try:
        if original_label in save_directories:
            save_path = save_generated_image(np_image,save_directories[original_label],image_idx,variant_idx)
        else:
            # Create directory for unknown class
            unknown_dir = create_save_directory(original_label, save_base_path)
            save_directories[original_label] = unknown_dir
            save_path = save_generated_image(np_image,unknown_dir,image_idx,variant_idx)
    except Exception as e:
        print(f"Failed to save variant {variant_idx} for image {image_idx}: {e}")


In [21]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [22]:
# Print prediction results
def print_stats(save_variants, original_label, save_directories, image_idx, variant_idx, predicted_class_name, predicted_class, save_color_maps):
    if save_variants and original_label in save_directories:
        image_name = f"image_{image_idx:04d}_variant_{variant_idx:02d}.png"
    else:
        image_name = f"image_{image_idx:04d}_variant_{variant_idx:02d}"

    # Check if prediction matches original label
    is_correct = predicted_class_name == original_label
    status = "CORRECT" if is_correct else "WRONG PREDICTION"

    colormap_info = ""
    if save_color_maps:
        colormap_name = f"colormap_{image_idx:04d}_variant_{variant_idx:02d}.png"
        colormap_info = f" | ColorMap: {colormap_name}"
    print(f"{status} - {image_name} : prediction : {predicted_class} {predicted_class_name}{colormap_info}")



In [23]:
def extract_orig_labels_and_pred_labels(dataset_holder):
    original_labels = []
    predicted_labels = []

    for image in dataset_holder.get_image_generator():
        original_label = process_label(image.label)
        print("original_label", original_label)
        # Add original label for each variant prediction
        for pred_label in image.predicted_variants_labels:
            processed_pred = process_label(pred_label)
            print("original_labels", original_labels)
            original_labels.append(original_label)
            print("predicted_labels", predicted_labels)
            predicted_labels.append(processed_pred)
    print("original_labels, predicted_labels", original_labels, predicted_labels)
    return original_labels, predicted_labels

In [24]:
def calc_all_metrics(original_labels, predicted_labels, robustness_threshold = 0.85):
    metrics = {
        'accuracy': accuracy_score(original_labels, predicted_labels),
        'precision': precision_score(original_labels, predicted_labels, average='weighted', zero_division=0),
        'recall': recall_score(original_labels, predicted_labels, average='weighted', zero_division=0),
        'f1_score': f1_score(original_labels, predicted_labels, average='weighted', zero_division=0),
        'confusion_matrix': confusion_matrix(original_labels, predicted_labels)
    }

    # Robustness evaluation
    is_robust = metrics['accuracy'] >= robustness_threshold

    all_labels = list(set(original_labels + predicted_labels))
    all_labels = list(set(all_labels))
    all_labels.sort()

    return metrics, is_robust, all_labels

In [25]:
def get_optional_metrics(dataset_holder, all_labels, original_labels, predicted_labels):
    consistency_scores = []
    for image in dataset_holder.get_image_generator():
        if image.predicted_variants_labels:
            processed_preds = [process_label(pred) for pred in image.predicted_variants_labels]
            consistency = len(set(processed_preds)) == 1
            consistency_scores.append(consistency)

    overall_consistency = np.mean(consistency_scores) if consistency_scores else 0

    # Per-class analysis
    class_accuracy = {}
    for class_name in all_labels:
        class_indices = [i for i, label in enumerate(original_labels) if label == class_name]
        if class_indices:
            class_predictions = [predicted_labels[i] for i in class_indices]
            class_correct = sum(1 for pred in class_predictions if pred == class_name)
            class_accuracy[class_name] = class_correct / len(class_predictions)
        else:
            class_accuracy[class_name] = 0.0
    return overall_consistency, class_accuracy

In [26]:
import pandas as pd

# Read file and create lookup dictionary
df = pd.read_csv("./words.txt", sep='\t', header=None, names=['id', 'class_name'])
id_to_class = dict(zip(df['id'], df['class_name']))

def get_class_name(class_id):
    return id_to_class.get(class_id, "ID not found")

# Example
print(get_class_name("n00020090"))

substance


## New

In [27]:
import os
from PIL import Image
import numpy as np

def evaluate_model_robustness(
    model: Any,
    dataset_path: str,
    prompt: str ='filled later',
    class_names: Optional[List[str]] = None,
    num_variants: int = 3,
    num_blocks: int = 4,
    canny_low: int = 100,
    canny_high: int = 200,
    blend_alpha: float = 0.7,
    robustness_threshold: float = 0.85,
    save_variants: bool = False,
    save_base_path: str = "./variants",
    save_color_maps: bool = False,
    color_maps_base_path: str = "./color_maps",
    verbose: bool = True
) -> Dict[str, Any]:


    all_original_labels = []
    all_predicted_labels = []


    # Initialize dataset first to extract class names if needed
    print("Initializing dataset...")
    dataset_holder = ImageDataset(dataset_path=dataset_path)
    # Auto-detect class names if not provided
    print("Auto-detecting class names from dataset...")

    class_names = extract_class_names_from_dataset_object(dataset_holder)
    # print(f"Class names extracted from dataset object: {class_names}")

    # Create save directories if needed
    save_directories = {}
    colormap_directories = {}
    if save_variants:
        print(f"Creating save directories at: {save_base_path}")
        for class_name in class_names:
            save_directories[class_name] = create_save_directory(class_name, save_base_path)
        print(f"Created {len(save_directories)} class directories for saving variants")

    if save_color_maps:
        print(f"Creating color map directories at: {color_maps_base_path}")
        for class_name in class_names:
            colormap_directories[class_name] = create_save_directory(class_name, color_maps_base_path)
        print(f"Created {len(colormap_directories)} class directories for saving color maps")


    print(f"Dataset loaded: {len(dataset_holder)} images", f"Detected {len(class_names)}", "Generating variants and making predictions...")

    # Process each image and generate variants
    processed_images = 0
    total_variants = 0
    saved_images_count = 0
    saved_colormaps_count = 0

    for image_idx, image in enumerate(dataset_holder.get_image_generator()):
        # Get original label for this image
        original_label = process_label(image.label)
        # Generate color conditions
        color_condition_creator = ColorCannyConditionCreator(image=image)
        color_maps = color_condition_creator.generate_multiple_conditions()
        image.extend_color_maps(color_maps)

        # Generate variants and make predictions
        for variant_idx, color_map in enumerate(image.color_maps):
            prompt=image.caption
            prompt = prompt + f". The object in the image is {get_class_name(original_label)}"
            print(prompt)
            # Save color map if requested
            if save_color_maps:
                save_color_map_np(color_map, colormap_directories, original_label, image_idx, variant_idx, color_maps_base_path)

            # Generate variant image
            generated_image = diffuser.generate_fused_image(
                input_image=color_map,
                prompt=prompt + f". The object in the image is {get_class_name(original_label)}",
            )
            print('The prompt is: ',prompt)
            # Convert to numpy array for storage
            np_image = np.array(generated_image)
            # Save variant image if requested
            if save_variants:
                save_img_variant(original_label, save_directories, image_idx, variant_idx, np_image)

            # Store variant in ImageContainer class
            image.add_variants(np_image)

            # Convert to tensor for prediction
            pil_image = Image.fromarray(np_image.astype('uint8'))
            tensor_image = transform(pil_image).unsqueeze(0)

            # Make prediction and store
            prediction = model_holder.predict(input_data=tensor_image)
            predicted_class = torch.argmax(prediction, dim=1).item()
            predicted_class_name = class_names[predicted_class]
            print("appending", image.predicted_variants_labels)
            image.predicted_variants_labels.append(predicted_class_name)
            all_original_labels.append(process_label(original_label))
            all_predicted_labels.append(process_label(predicted_class_name))
            print("appending", image.predicted_variants_labels)

            print_stats(save_variants, original_label, save_directories, image_idx, variant_idx, predicted_class_name, predicted_class, save_color_maps)

        processed_images += 1
        if processed_images % 10 == 0:
            print(f"Processed {processed_images}/{len(dataset_holder)} images...")

    # Collect all original and predicted labels
    original_labels, predicted_labels = all_original_labels, all_predicted_labels



    # Calculate comprehensive metrics
    metrics, is_robust, all_labels = calc_all_metrics(original_labels, predicted_labels)

    # Per-image consistency analysis
    overall_consistency, class_accuracy = get_optional_metrics(dataset_holder, all_labels, original_labels, predicted_labels)

    # Compile results
    results = {
        'metrics': metrics,
        'robustness': {
            'is_robust': is_robust,
            'threshold': robustness_threshold,
            'accuracy': metrics['accuracy']
        },
        'consistency': {
            'overall_consistency': overall_consistency,
        },
        'class_analysis': {
            'per_class_accuracy': class_accuracy,
            'detected_classes': all_labels,
            'class_distribution': {label: original_labels.count(label) for label in all_labels}
        },
        'dataset_info': {
            'total_images': len(dataset_holder),
            'total_variants': total_variants,
            'variants_per_image': num_variants,
            'class_names': class_names,
            'detected_classes': all_labels
        },
        'saving_info': {
            'save_variants_enabled': save_variants,
            'saved_images_count': saved_images_count if save_variants else 0,
            'save_base_path': save_base_path if save_variants else None,
            'save_directories': list(save_directories.keys()) if save_variants else [],
            'save_color_maps_enabled': save_color_maps,
            'saved_colormaps_count': saved_colormaps_count if save_color_maps else 0,
            'color_maps_base_path': color_maps_base_path if save_color_maps else None,
            'colormap_directories': list(colormap_directories.keys()) if save_color_maps else []
        }
    }

    # Display results if verbose
    print("\n" + "="*50)
    print("EVALUATION RESULTS")
    print("="*50)
    print(f"Total images processed: {results['dataset_info']['total_images']}")
    print(f"Total variants generated: {results['dataset_info']['total_variants']}")

    print(f"Total variant images saved: {saved_images_count}")
    print(f"Save location: {save_base_path}")
    print(f"Class directories created: {len(save_directories)}")

    print(f"Total color maps saved: {saved_colormaps_count}")
    print(f"Color maps location: {color_maps_base_path}")
    print(f"Color map directories created: {len(colormap_directories)}")

    print(f"Detected classes: {len(all_labels)} - {all_labels}")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1_score']:.4f}")
    print(f"Model is {'ROBUST' if is_robust else 'NOT ROBUST'} (threshold: {robustness_threshold})")

    print("\n" + "-"*30)
    print("CONSISTENCY ANALYSIS")
    print("-"*30)
    print(f"Overall prediction consistency: {overall_consistency:.4f}")

    print("\n" + "-"*30)
    print("PER-CLASS ACCURACY")
    print("-"*30)
    for class_name, accuracy in class_accuracy.items():
        count = results['class_analysis']['class_distribution'].get(class_name, 0)
        print(f"{class_name}: {accuracy:.4f} ({count} samples)")
    return results

In [28]:
def save_generated_image(image_array: np.ndarray, save_path: str, image_idx: int, variant_idx: int) -> str:
        """Save generated image to specified path"""
        # Ensure image is in correct format
        if image_array.dtype != np.uint8:
            # Normalize to 0-255 range if needed
            if image_array.max() <= 1.0:
                image_array = (image_array * 255).astype(np.uint8)
            else:
                image_array = np.clip(image_array, 0, 255).astype(np.uint8)

        # Convert to PIL Image
        if len(image_array.shape) == 3:
            if image_array.shape[0] in [1, 3, 4]:  # Channel first
                image_array = np.transpose(image_array, (1, 2, 0))
            pil_image = Image.fromarray(image_array)
        else:
            pil_image = Image.fromarray(image_array)

        # Create filename
        filename = f"image_{image_idx:04d}_variant_{variant_idx:02d}.png"
        full_path = os.path.join(save_path, filename)

        # Save image
        pil_image.save(full_path)
        return full_path

In [ ]:
import shutil
import os

folder_to_delete = "./lim25-imagenet-image-small-3/.ipynb_checkpoints"

if os.path.exists(folder_to_delete):
    shutil.rmtree(folder_to_delete)
    print(f"Deleted: {folder_to_delete}")
else:
    print(f"Folder not found: {folder_to_delete}")
    
results = evaluate_model_robustness(
    model=model,
    dataset_path="./lim25-imagenet-image-small-3",
    class_names=None,  # Will be auto-detected
    num_variants=2,
    robustness_threshold=0.85,
    verbose=True,
    save_variants=True,
    save_color_maps = True
)

# Access results
print(f"Model accuracy: {results['metrics']['accuracy']:.4f}")
print(f"Is robust: {results['robustness']['is_robust']}")
print(f"Consistency: {results['consistency']['overall_consistency']:.4f}")
print(f"Detected classes: {results['dataset_info']['detected_classes']}")


Folder not found: ./lim25-imagenet-image-small-3/.ipynb_checkpoints
Initializing dataset...
Indexed dataset with 1000 labels and 3000 total images
Auto-detecting class names from dataset...
Creating save directories at: ./variants
Created 1000 class directories for saving variants
Creating color map directories at: ./color_maps
Created 1000 class directories for saving color maps
Dataset loaded: 3000 images Detected 1000 Generating variants and making predictions...
a man holding a fish in his hand. The object in the image is tench, Tinca tinca
Generating from control image


  0%|          | 0/50 [00:00<?, ?it/s]

The prompt is:  a man holding a fish in his hand. The object in the image is tench, Tinca tinca
appending []
appending ['n01440764']
CORRECT - image_0000_variant_00.png : prediction : 0 n01440764 | ColorMap: colormap_0000_variant_00.png
a man holding a fish in his hand. The object in the image is tench, Tinca tinca
Generating from control image


  0%|          | 0/50 [00:00<?, ?it/s]

The prompt is:  a man holding a fish in his hand. The object in the image is tench, Tinca tinca
appending ['n01440764']
appending ['n01440764', 'n03908618']
WRONG PREDICTION - image_0000_variant_01.png : prediction : 709 n03908618 | ColorMap: colormap_0000_variant_01.png
a man holding a fish in his hand. The object in the image is tench, Tinca tinca
Generating from control image


...

  0%|          | 0/50 [00:00<?, ?it/s]

The prompt is:  a bird is standing on top of a wooden structure. The object in the image is Bouvier des Flandres, Bouviers des Flandres
appending []
appending ['n02124075']
WRONG PREDICTION - image_0701_variant_00.png : prediction : 285 n02124075 | ColorMap: colormap_0701_variant_00.png
a bird is standing on top of a wooden structure. The object in the image is Bouvier des Flandres, Bouviers des Flandres
Generating from control image


  0%|          | 0/50 [00:00<?, ?it/s]

The prompt is:  a bird is standing on top of a wooden structure. The object in the image is Bouvier des Flandres, Bouviers des Flandres
appending ['n02124075']
appending ['n02124075', 'n03376595']
WRONG PREDICTION - image_0701_variant_01.png : prediction : 559 n03376595 | ColorMap: colormap_0701_variant_01.png
a bird is standing on top of a wooden structure. The object in the image is Bouvier des Flandres, Bouviers des Flandres
Generating from control image


  0%|          | 0/50 [00:00<?, ?it/s]

In [30]:
print(f"Model accuracy: {results['metrics']['accuracy']:.4f}")
print(f"Is robust: {results['robustness']['is_robust']}")
print(f"Consistency: {results['consistency']['overall_consistency']:.4f}")
print(f"Detected classes: {results['dataset_info']['detected_classes']}")

Model accuracy: 0.2802
Is robust: False
Consistency: 0.0000
Detected classes: ['n01440764', 'n01443537', 'n01484850', 'n01491361', 'n01494475', 'n01496331', 'n01498041', 'n01514668', 'n01514859', 'n01518878', 'n01530575', 'n01531178', 'n01532829', 'n01534433', 'n01537544', 'n01558993', 'n01560419', 'n01580077', 'n01582220', 'n01592084', 'n01601694', 'n01608432', 'n01614925', 'n01616318', 'n01622779', 'n01629819', 'n01630670', 'n01631663', 'n01632458', 'n01632777', 'n01641577', 'n01644373', 'n01644900', 'n01664065', 'n01665541', 'n01667114', 'n01667778', 'n01669191', 'n01675722', 'n01677366', 'n01682714', 'n01685808', 'n01687978', 'n01688243', 'n01689811', 'n01692333', 'n01693334', 'n01694178', 'n01695060', 'n01697457', 'n01698640', 'n01704323', 'n01728572', 'n01728920', 'n01729322', 'n01729977', 'n01734418', 'n01735189', 'n01737021', 'n01739381', 'n01740131', 'n01742172', 'n01744401', 'n01748264', 'n01749939', 'n01751748', 'n01753488', 'n01755581', 'n01756291', 'n01768244', 'n01770081'

In [32]:
import shutil
import os

folder_path = './variants'

# Check if the folder exists before deleting
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    shutil.rmtree(folder_path)
    print(f"Deleted folder: {folder_path}")
else:
    print(f"Folder does not exist: {folder_path}")

import shutil
import os

folder_path = './color_maps'

# Check if the folder exists before deleting
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    shutil.rmtree(folder_path)
    print(f"Deleted folder: {folder_path}")
else:
    print(f"Folder does not exist: {folder_path}")
